In [1]:
# Standard Libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, label_binarize
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc, precision_recall_curve, average_precision_score, classification_report

# TensorFlow/Keras Models and Layers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, GRU, Dense, Conv1D, MaxPooling1D, Flatten, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.python.client import device_lib

# Transformer Libraries
from transformers import TFAutoModel, AutoTokenizer


In [2]:
file_path = r"C:\\Users\\Amirreza\\Downloads\\Documents\\Project\\LSTM Model\\Data\\all_data.csv"
df = pd.read_csv(file_path)

In [3]:
def preprocess_data(df, sequence_length):
    # drop unwanted column
    # df = df.drop(columns=['P-JUS-CKGL', 'T-JUS-CKGL', 'QGL'])

    # convert timestamp to datetime and extract features
    df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S.%f')
    df['hour'] = df['timestamp'].dt.hour
    df['minute'] = df['timestamp'].dt.minute
    df['second'] = df['timestamp'].dt.second
    df['seconds_since_midnight'] = df['hour'] * 3600 + df['minute'] * 60 + df['second']
    df = df.drop(columns=['timestamp', 'hour', 'minute', 'second'])

    # Assume 'text_column' is your text data to be processed by BERT
    texts = df['text_column'].tolist()
    #separate features and target
    X = df.drop(columns=['class', 'text_column'])
    y = df['class']

    # preprocessing pipeline for numeric features
    numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
    numeric_transformer = StandardScaler()

    # Preprocessing pipeline for categorical features
    categorical_features = X.select_dtypes(include=['object', 'category']).columns
    categorical_transformer = OneHotEncoder(handle_unknown='ignore')

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('car', categorical_transformer, categorical_features)
        ]
    )

    # apply transformation 
    X_processed = preprocessor.fit_transform(X)

    # encode class labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    # Initialize lists for sequences
    X_sequences = []
    y_sequences = []


    # Create sequences
    for i in range(len(X_processed) - sequence_length):
        X_sequences.append(X_processed[i:i + sequence_length])
        y_sequences.append(y_encoded[i + sequence_length])

    # create sequences
    X_sequences = np.array(X_sequences)
    y_sequences = np.array(y_sequences)

    return texts, X_sequences, y_sequences  # Return texts for BERT processing

In [4]:
def create_transformer_model():
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    transformer_model = TFAutoModel.from_pretrained("bert-base-uncased")
    
    # Define model architecture
    input_text = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text_input')
    tokenized = tokenizer(input_text, return_tensors='tf', padding=True, truncation=True, max_length=512)
    transformer_output = transformer_model(tokenized['input_ids'])
    sequence_output = transformer_output.last_hidden_state
    cls_token = sequence_output[:, 0, :]  # Use the CLS token for classification
    out = tf.keras.layers.Dense(15, activation='softmax')(cls_token)
    
    model = tf.keras.Model(inputs=input_text, outputs=out)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

In [5]:
# Lists to store results
acc_scores = []
val_acc_scores = []
test_accuracy = []

# Define sequence length
sequence_length = 60   # 1 hours

# preprocess the data
texts, X, y = preprocess_data(df, sequence_length)

# Tokenize the text data
tokenized_texts = tokenizer(texts, return_tensors='tf', padding=True, truncation=True, max_length=512)

# Split data into train, validation, and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.05/(1-0.05), random_state=42)

# Create a MirroredStrategy.
strategy = tf.distribute.MirroredStrategy()

# Open a strategy scope.
with strategy.scope():
    # Create the model inside the strategy's scope.
    model = create_transformer_model()

    # Define the early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Train the model
    history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])

    # Evaluate on the test set
    test_loss, test_accuracy = model.evaluate(X_test, y_test)

# Store the results
acc_scores.append(history.history['accuracy'])
val_acc_scores.append(history.history['val_accuracy'])
test_accuracy.append(test_accuracy)


KeyError: 'text_column'

In [7]:
# Print the results
print("Test Accuracy:", test_accuracy)

Test Accuracy: 0.9751362204551697


In [8]:
model.save("Transformer_final_model.h5")

c:\Users\Amirreza\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [9]:

model1 = load_model("Transformer_final_model.h5")

In [ ]:
# styling for plots
sns.set(style="whitegrid")
plt.style.use('ggplot')

In [ ]:
# Confusion Matrix
def plot_confusion_matrix(y_test , y_pred, classes):
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='blues', xticklabels=classes, yticklabels=classes)
    plt.title(f'Transformer_confusion_matrix')
    plt.ylabel('True label')
    plt.xlabel('predicted label')
    plt.savefig(f'Transformer_confusion_matrix.png')
    plt.show()

In [ ]:
# ROC Curve and AUC
def plot_roc_curve(y_test, y_score, n_classes):
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    
    for i in range (n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test[:,i], y_score[:,i])
        roc_auc[i] = auc(fpr[i], tpr[i])
        
    plt.figure(figsize=(10,7))
    for i in range(n_classes):
        plt.plot(fpr[i], tpr[i], lw=2, label='class{0} (area = {1:0.2f})'.format(i, roc_auc))
        
    plt.plot([0,1], [0,1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'Transformer - Receivor Operating Characteristic (ROC)')
    plt.legend(loc="lower right")
    plt.savefig(f'Transformer_roc_curve.png')
    plt.show()

In [ ]:
# Precision-Recall Curve
def plot_precision_recall_curve(y_test, y_score, n_classes):
    precision = dict()
    recall = dict()
    average_precision = dict()
    
    for i in range (n_classes):
        precision[i], recall[i], _ = precision_recall_curve(y_test[:,i], y_score[:,i])
        average_precision[i] = average_precision_score(y_test[:,i], y_score[:,i])
        
    plt.figure(figsize=(10,7))
    for i in range (n_classes):
        plt.plot(recall[i], precision[i], lw=2, label='Class[0] (area = {1:0.2f})'.format(i, average_precision[i]))
        
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'Transformer - precision-recall curve')
    plt.legend(loc="best")
    plt.savefig(f'Transformer_precision_recall_curve.png')
    plt.show()
        

In [2]:
# Training and Validation plots
def plot_training_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(acc) + 1)
    
    # accuracy plot 
    plt.figure(figsize=(7,5))
    plt.plot(epochs, acc, 'bo-', label='Training accuracy')
    plt.plot(epochs, val_acc, 'r*-', label='Validation accuracy')
    plt.title('Transformer - Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.savefig(f'Transformer_training_accuracy')
    
    
    # Loss plot 
    plt.figure(figsize=(7,5))
    plt.plot(epochs, loss, 'bo-', label='Training loss')
    plt.plot(epochs, val_loss, 'r*-', label='Validation loss')
    plt.title('Transformer - Training and validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(f'Transformer_training_loss')
    plt.show()



In [ ]:
# Generate classification report
def save_classification_report(y_test, y_pred, classes):
    report = classification_report(y_test, y_pred, target_names=classes)
    print(report)
    with open(f"Transformer_classification_report.txt", "w") as f:
        f.write(report)

In [ ]:
classes = [str(i) for i in range(15)] 

# Predict on test data
y_pred = np.argmax(model1.predict(X_test), axis=-1)
y_score = model1.predict(X_test)

# Binarize the output for multiclass metrics
y_test_bin = label_binarize(y_test, classes=range(len(classes)))

# Plot and save all the metrics
plot_training_history(history)
plot_confusion_matrix(y_test, y_pred, classes)
plot_roc_curve(y_test_bin, y_score, len(classes))
plot_precision_recall_curve(y_test_bin, y_score, len(classes))
save_classification_report(y_test, y_pred, classes)